In [6]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
import numpy as np

In [18]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Convert to float32.
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

# Flatten images to 1-D vector of 784 features (28*28).
num_features=784
print(x_train.shape)

x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
print(x_train.shape)

# Normalize images value from [0, 255] to [0, 1].
x_train, x_test = x_train / 255., x_test / 255.

(60000, 28, 28)
(60000, 784)


In [19]:
num_classes = 10 
num_features = 784 
learning_rate = 0.01
training_steps = 1000
batch_size = 256
display_step = 50

In [9]:
train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_data=train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)
print(train_data)

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 784), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>


In [11]:
W = tf.Variable(tf.ones([num_features, num_classes]), name="weight")
print(W)

b = tf.Variable(tf.zeros([num_classes]), name="bias")
print(b)

<tf.Variable 'weight:0' shape=(784, 10) dtype=float32, numpy=
array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)>
<tf.Variable 'bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>


In [17]:
def logistic_regression(x):
  # Apply softmax to normalize the logits to a probability distribution.
  sm=tf.nn.softmax(tf.matmul(x, W) + b)
  return sm

def cross_entropy(y_pred, y_true):

  # Encode label to a one hot vector.
  y_true = tf.one_hot(y_true, depth=num_classes)

  # Clip prediction values to avoid log(0) error.
  y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
  
  # Compute cross-entropy
  entropy=tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))
  return entropy

In [13]:
def accuracy(y_pred, y_true):
  correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
  acc=tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  return acc

optimizer = tf.optimizers.SGD(learning_rate)

In [21]:
def accuracy(y_pred, y_true):
  
  # Predicted class is the index of the highest score in prediction vector (i.e. argmax).
  correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
  acc=tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  return acc

optimizer = tf.optimizers.SGD(learning_rate)

In [15]:
def run_optimization(x, y):
  
  # Wrap computation inside a GradientTape for automatic differentiation.
  with tf.GradientTape() as g:
    pred = logistic_regression(x)
    loss = cross_entropy(pred, y)
    
  # Compute gradients.
  gradients = g.gradient(loss, [W, b])

  # Update W and b following gradients.
  optimizer.apply_gradients(zip(gradients, [W, b]))

In [23]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
  
    # Run the optimization to update W and b values.
    run_optimization(batch_x, batch_y)
    if step % display_step == 0:
        pred = logistic_regression(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 50, loss: 493.495117, accuracy: 0.859375
step: 100, loss: 690.879822, accuracy: 0.777344
step: 150, loss: 444.488373, accuracy: 0.855469
step: 200, loss: 461.691803, accuracy: 0.878906
step: 250, loss: 722.393555, accuracy: 0.777344
step: 300, loss: 647.897644, accuracy: 0.832031
step: 350, loss: 517.229919, accuracy: 0.843750
step: 400, loss: 846.312805, accuracy: 0.792969
step: 450, loss: 658.825439, accuracy: 0.832031
step: 500, loss: 524.262512, accuracy: 0.867188
step: 550, loss: 458.550446, accuracy: 0.875000
step: 600, loss: 657.968201, accuracy: 0.847656
step: 650, loss: 619.149719, accuracy: 0.843750
step: 700, loss: 640.463013, accuracy: 0.832031
step: 750, loss: 547.974731, accuracy: 0.843750
step: 800, loss: 439.055145, accuracy: 0.820312
step: 850, loss: 330.447327, accuracy: 0.906250
step: 900, loss: 589.210754, accuracy: 0.808594
step: 950, loss: 645.736816, accuracy: 0.839844
step: 1000, loss: 535.106506, accuracy: 0.867188


In [26]:
pred = logistic_regression(x_test)
print("Test Accuracy: %f" % accuracy(pred, y_test))

Test Accuracy: 0.837300
